In [1]:
import pandas as pd

In [2]:
import nettoyage_lib as nl

In [3]:
donnees_brutes = pd.read_csv("https://raw.githubusercontent.com/VPerrollaz/immobilier/master/donnees/data.tsv", sep="\t")

In [4]:
donnees_nettoyees = (
    donnees_brutes
    .pipe(nl.conversion_types)  
    .pipe(nl.suppression_annonces_redondantes)
    .pipe(nl.suppression_colonne_id)
    .pipe(nl.numerise_les_colonnes)
    .pipe(nl.supprime_partiellement_na)
)

In [5]:
donnees_nettoyees.head()

,Genre,Neuf,Surface,Pieces,Quartier,Prix
0,0,0,90.00,3,cathédrale,374400
1,0,0,146.27,5,sud,499200
2,0,0,110.00,5,prébendes,499200
3,1,0,132.00,6,prébendes,508000
4,1,0,185.00,7,strasbourg,676000


# Rappel sur le projet en cours

1. Problématique : identifier les bonnes annonces immobilières sur Tours 
2. Modélisation : en utilisant un prédicteur de prix représentant la connaissance du marché immobilier, les bonnes annonces sont celles dont le prix est en dessous de ce qui est prédit à partir des caractéristiques.
3. Analyse du prédicteur : jeu de données disponible sur internet, les annonces de seloger.com.
4. Analyse de la faisabilité : 1600 annonces avec facilement accessibles prix, nombre de pièces, surface, type de logement (genre et ancien) et plus délicatement le quartier.
5. Scraping des données via selenium : on extrait directement ce qui est facilement accessible et on stocke les infos brutes dont on pourra extraire le quartier; resultat, un fichier json.
6. Transformation : utilisation de regex (entre autres) pour extraire un quartier et rendre les données tabulaires; resultat, un fichier tsv.
7. Nettoyage : préparation des données via pandas; resultat, un pipeline pandas pour transformer le tableau avant de le passer à sklearn.
8. Apprentissage : Entrainement et sélection d'un prédicteur; resultat, pipeline sklearn.
9. Conclusion : utiliser le prédicteur pour répondre à la question originale.

**REMARQUE** : A chaque étape, avant de passer à la suivante, on nettoie un maximum le code pour pouvoir le reprendre facilement plus tard si le besoin s'en fait sentir. Et pour pouvoir plus facilement communiquer dessus en phase finale.

# Phase exploratoire avec sklearn

1. Sans la colonne quartier, entrainer un opérateur de prédiction des prix, en faisant la cross-validation pour la algorithmes pertinent.
2. Ajouter un LabelEncoder pour Quartier et faites de même.
3. Utiliser un OneHotEncoder et faites de même.
4. Etudier l'impact de scaler en entrée en faisant des Pipeline.

# Version Basique sans le Quartier

On a à faire à un problème de régression. Les algorithmes que l'on va considérer sont donc
- Régression linéaire (et les dérivées : ridge, lasso, elasticnet)
- Plus proches voisins.
- Processus Gaussiens.
- Support Vecteur Machine
- Arbres de décisions et Forêts aléatoires.
- Réseaux de neurones. (Attention il peut y avoir besoin de beaucoup plus de données)

In [7]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

On aura besoin de faire une séparation du jeu de données, et de validation croisée.

In [8]:
from sklearn.model_selection import train_test_split, cross_val_score

## A FAIRE

- Séparer le jeu de données.
- Identifier les hyperparamètres des différents algorithmes. (on consultera avec bonheur la documentation de `scikit-learn`)
- Construire l'ensemble des estimateurs.
- Faire la validation croisée. (On pourra aussi regarder `GridSearchCV` et `RandomSearchCV`)
- Entrainer le prédicteur final et le valider.

In [11]:
X = donnees_nettoyees[["Genre", "Neuf", "Surface", "Pieces"]].values
X.shape

(1625, 4)

In [12]:
y = donnees_nettoyees.Prix.values
y.shape

(1625,)

In [13]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, random_state=42)

In [14]:
modeles = list()
modeles.append(LinearRegression())

In [15]:
for val_alpha in (1e-3, 1e-2, 1e-1, 1):
    modeles.append(Lasso(alpha=val_alpha))

In [16]:
for val_alpha in (1e-3, 1e-2, 1e-1, 1):
    modeles.append(Ridge(alpha=val_alpha))

In [17]:
for val_alpha in (1e-3, 1e-2, 1e-1, 1):
    for val_l1 in (0.25, 0.5, 0.75):
        modeles.append(ElasticNet(alpha=val_alpha, l1_ratio=val_l1))

In [18]:
for nb_voisins in range(3, 10):
    modeles.append(KNeighborsRegressor(n_neighbors=nb_voisins))

In [19]:
modeles.append(GaussianProcessRegressor())

In [20]:
for val_epsilon in (10 ** n for n in range(-3, 1)):
    for val_C in (10 ** n for n in range(-3, 4)):
        modeles.append(SVR(epsilon=val_epsilon, C=val_C))

In [21]:
for nb_estimateurs in (50, 100, 150, 200):
    modeles.append(RandomForestRegressor(n_estimators=nb_estimateurs))

In [22]:
for nb_neurones in  ((100,), (50, 50), (25, 50, 25)):
    modeles.append(MLPRegressor(hidden_layer_sizes=nb_neurones))

In [23]:
resultats = dict()
for modele in modeles:
    resultats[modele] = cross_val_score(modele, X_tr, y_tr, cv=5)
    

/home/vincent/miniconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/vincent/miniconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/vincent/miniconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/vincent/miniconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/vincent/miniconda3/lib/pyt

In [32]:
for modele, score in resultats.items():
    print(f"{repr(modele):50} : {score.mean()}, {score.std()}")

LinearRegression()                                 : 0.5617034498529294, 0.25033702218722426
Lasso(alpha=0.001)                                 : 0.5617034564975223, 0.25033701258725877
Lasso(alpha=0.01)                                  : 0.5617035161766017, 0.25033692639723676
Lasso(alpha=0.1)                                   : 0.5617041156475319, 0.2503360593296961
Lasso(alpha=1)                                     : 0.5617100959626219, 0.25032741188047275
Ridge(alpha=0.001)                                 : 0.5617035618739867, 0.2503369646205551
Ridge(alpha=0.01)                                  : 0.5617045699027593, 0.250336446552092
Ridge(alpha=0.1)                                   : 0.5617146343009024, 0.2503312689881091
Ridge(alpha=1)                                     : 0.5618137124034834, 0.2502798035195504
ElasticNet(alpha=0.001, l1_ratio=0.25)             : 0.5617843882614041, 0.2502950975304494
ElasticNet(alpha=0.001)                            : 0.561757619273975, 0.250

In [36]:
resultats_pour_tri = sorted([(scores.mean(), scores.std(), repr(modele)) for modele, scores in resultats.items()], reverse=True)

In [39]:
for moyenne, ecart_type, nom_modele in resultats_pour_tri:
    print(f"{nom_modele:45} {moyenne:4.3}   {ecart_type:6.5}")

KNeighborsRegressor(n_neighbors=9)            0.67   0.12259
KNeighborsRegressor(n_neighbors=8)            0.667   0.12309
KNeighborsRegressor(n_neighbors=7)            0.66   0.12376
KNeighborsRegressor()                         0.652   0.11808
KNeighborsRegressor(n_neighbors=6)            0.65   0.11755
KNeighborsRegressor(n_neighbors=4)            0.626   0.12248
MLPRegressor(hidden_layer_sizes=(25, 50, 25)) 0.62   0.17447
RandomForestRegressor()                       0.596   0.11907
KNeighborsRegressor(n_neighbors=3)            0.592   0.11032
RandomForestRegressor(n_estimators=200)       0.592   0.12911
RandomForestRegressor(n_estimators=150)       0.59   0.11352
RandomForestRegressor(n_estimators=50)        0.59   0.11783
ElasticNet(alpha=1, l1_ratio=0.25)            0.575   0.24123
ElasticNet(alpha=1)                           0.572   0.24237
ElasticNet(alpha=1, l1_ratio=0.75)            0.569   0.24436
ElasticNet(alpha=0.1, l1_ratio=0.25)          0.566   0.2474
ElasticNet(alph

## Bilan:
- Pour les voisins on a l'impression que plus on en rajoute mieux c'est, donc on pourrait tester avec plus de voisins encore.
- Pour les processus gaussiens il faut clairement recaler les hyperparamètres.
- Les réseaux de neurones sont partis dans le décors dans la majorité des cas, il faudrait augmenter le nombre d'itérations de la minisation. (augementer `max_iter`) 
- On a des grosses fluctuations sur les supports vecteurs là encore on peut essayer de mieux ajuster les hyperparamètres.
- On pourra essayer de faire par type d'estimateur `RandomizedSearchCV`

In [44]:
# Idée pour ce que fait RandomizedSearch
import random

for _ in range(10):
    choix_param = random.expovariate(10)
    print(choix_param)
    modeles.append(Lasso(alpha=choix_param))

0.0287852152260116
0.004309237569341786
0.051770990393546434
0.014226773074174517
0.10213474456607763
0.22179298651002544
0.06757702863943502
0.028194414838754078
0.22611456179233486
0.04252295058980911


# AVANCER SUR CES PROBLEMATIQUES POUR LA SEMAINE PROCHAINE